In [202]:
import numpy as np
import pandas as pd
import xgboost
import catboost
import seaborn as sns
from sklearn.cross_validation import train_test_split
from sklearn.metrics import precision_score, recall_score

import warnings
warnings.simplefilter("ignore")

In [4]:
data_hour = pd.read_csv('./data_hw_2/data_hour_train')
data_action = pd.read_csv('./data_hw_2/data_action_train')

In [5]:
# obj_id - id объекта для склейки с событиями из data_action
data_hour.head()

,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,factor_8,factor_9,...,factor_308,factor_309,factor_310,factor_311,factor_312,factor_313,factor_314,factor_315,obj_id,target
0,900.0,0.0,NaN,2.0,NaN,NaN,NaN,990.0,NaN,1000.0,...,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,0,1
1,990.0,0.0,0.0,2.0,0.0,0.0,0.0,1000.0,0.0,250.0,...,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,1,1
2,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,2,1
3,1000.0,1000.0,NaN,2.0,NaN,NaN,NaN,1000.0,NaN,500.0,...,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,3,1
4,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,4,1


In [6]:
# obj_id - id объекта для склейки с агрегациями из data_hour
data_action.head()

,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,factor_8,factor_9,...,factor_143,factor_144,factor_145,factor_146,factor_147,factor_148,factor_149,factor_150,obj_id,target
0,70,62,1000,343,1000,0,0.0,0,1,0,...,991,982,0,124,999,999,0,982,1621,0
1,70,57,1000,120,1000,0,0.0,0,0,0,...,989,982,0,45,999,999,0,982,9627,0
2,262,63,0,76,1000,8,0.0,227,1,469,...,989,982,0,0,999,999,41,982,15674,0
3,70,56,1000,653,1000,2,0.0,1,0,1,...,998,982,0,687,999,999,41,982,20118,0
4,70,56,1000,653,1000,2,0.0,1,0,1,...,998,982,0,687,999,999,41,982,20118,0


In [7]:
# переносим целевую переменную из разных систем
target_action = data_action.pivot_table(index='obj_id', values = 'target', aggfunc = ('sum', 'count')).reset_index()
target_hour = data_hour[['obj_id', 'target']]
target_hour.rename(columns = {'target':'hour_target'}, inplace=True)

data_hour = pd.merge(data_hour, target_action, on='obj_id')
data_action = pd.merge(data_action, target_hour, on='obj_id')

In [8]:
data_hour[:3]

,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,factor_8,factor_9,...,factor_310,factor_311,factor_312,factor_313,factor_314,factor_315,obj_id,target,count,sum
0,900.0,0.0,NaN,2.0,NaN,NaN,NaN,990.0,NaN,1000.0,...,NaN,1000.0,NaN,NaN,NaN,NaN,0,1,1,1
1,990.0,0.0,0.0,2.0,0.0,0.0,0.0,1000.0,0.0,250.0,...,NaN,1000.0,NaN,NaN,NaN,NaN,1,1,4,4
2,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,1000.0,NaN,NaN,NaN,NaN,2,1,2,0


In [9]:
data_action[:3]

,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,factor_8,factor_9,...,factor_144,factor_145,factor_146,factor_147,factor_148,factor_149,factor_150,obj_id,target,hour_target
0,70,62,1000,343,1000,0,0.0,0,1,0,...,982,0,124,999,999,0,982,1621,0,0
1,70,62,1000,343,1000,0,0.0,0,1,0,...,982,0,124,999,999,0,982,1621,0,0
2,70,62,1000,343,1000,0,0.0,0,1,0,...,982,0,124,999,999,0,989,1621,0,0


### Первая модель

In [10]:
train_data = data_action.groupby('obj_id').mean()
X = train_data.drop("hour_target", axis=1)
y = train_data['hour_target']

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
cb = catboost.CatBoostClassifier(iterations=500, max_depth=7,
                                 loss_function="Logloss",
                                 custom_metric=['Logloss', 'AUC']
                                )

Валидируем и подбираем лучшие параметры. По времени этот процесс занял не очень много времени, подбирал только 2 параметра: `iterations` и `max_depth`. Этого хватило. 

In [49]:
cb.fit(X=X_train, y=y_train, eval_set=(X_val, y_val))

0:	learn: 0.4787765	test: 0.4811315	best: 0.4811315 (0)	total: 58.8ms	remaining: 29.3s
1:	learn: 0.4020157	test: 0.4032087	best: 0.4032087 (1)	total: 96.1ms	remaining: 23.9s
2:	learn: 0.3329317	test: 0.3315078	best: 0.3315078 (2)	total: 134ms	remaining: 22.3s
3:	learn: 0.2863240	test: 0.2848319	best: 0.2848319 (3)	total: 173ms	remaining: 21.5s
4:	learn: 0.2590388	test: 0.2570729	best: 0.2570729 (4)	total: 209ms	remaining: 20.6s
5:	learn: 0.2252936	test: 0.2227554	best: 0.2227554 (5)	total: 246ms	remaining: 20.2s
6:	learn: 0.2104871	test: 0.2073013	best: 0.2073013 (6)	total: 306ms	remaining: 21.6s
7:	learn: 0.1986540	test: 0.1960215	best: 0.1960215 (7)	total: 355ms	remaining: 21.8s
8:	learn: 0.1903932	test: 0.1876726	best: 0.1876726 (8)	total: 421ms	remaining: 22.9s
9:	learn: 0.1846563	test: 0.1822901	best: 0.1822901 (9)	total: 489ms	remaining: 24s
10:	learn: 0.1769193	test: 0.1747286	best: 0.1747286 (10)	total: 570ms	remaining: 25.3s
11:	learn: 0.1712048	test: 0.1691095	best: 0.1691095

95:	learn: 0.1117597	test: 0.1205139	best: 0.1205139 (95)	total: 5.28s	remaining: 22.2s
96:	learn: 0.1115947	test: 0.1203496	best: 0.1203496 (96)	total: 5.31s	remaining: 22.1s
97:	learn: 0.1113734	test: 0.1201382	best: 0.1201382 (97)	total: 5.36s	remaining: 22s
98:	learn: 0.1110433	test: 0.1200187	best: 0.1200187 (98)	total: 5.4s	remaining: 21.9s
99:	learn: 0.1108905	test: 0.1199253	best: 0.1199253 (99)	total: 5.46s	remaining: 21.8s
100:	learn: 0.1105686	test: 0.1198584	best: 0.1198584 (100)	total: 5.53s	remaining: 21.9s
101:	learn: 0.1104038	test: 0.1196737	best: 0.1196737 (101)	total: 5.59s	remaining: 21.8s
102:	learn: 0.1103111	test: 0.1196798	best: 0.1196737 (101)	total: 5.68s	remaining: 21.9s
103:	learn: 0.1099624	test: 0.1196763	best: 0.1196737 (101)	total: 5.74s	remaining: 21.9s
104:	learn: 0.1098691	test: 0.1196262	best: 0.1196262 (104)	total: 5.8s	remaining: 21.8s
105:	learn: 0.1096422	test: 0.1195936	best: 0.1195936 (105)	total: 5.85s	remaining: 21.7s
106:	learn: 0.1094867	te

188:	learn: 0.0975459	test: 0.1143771	best: 0.1143520 (186)	total: 9.98s	remaining: 16.4s
189:	learn: 0.0975047	test: 0.1143956	best: 0.1143520 (186)	total: 10s	remaining: 16.4s
190:	learn: 0.0974454	test: 0.1144236	best: 0.1143520 (186)	total: 10.1s	remaining: 16.3s
191:	learn: 0.0973833	test: 0.1144756	best: 0.1143520 (186)	total: 10.1s	remaining: 16.2s
192:	learn: 0.0971161	test: 0.1144377	best: 0.1143520 (186)	total: 10.2s	remaining: 16.2s
193:	learn: 0.0969761	test: 0.1142987	best: 0.1142987 (193)	total: 10.2s	remaining: 16.1s
194:	learn: 0.0968686	test: 0.1142299	best: 0.1142299 (194)	total: 10.3s	remaining: 16.1s
195:	learn: 0.0966951	test: 0.1141600	best: 0.1141600 (195)	total: 10.3s	remaining: 16s
196:	learn: 0.0965744	test: 0.1139944	best: 0.1139944 (196)	total: 10.3s	remaining: 15.9s
197:	learn: 0.0964598	test: 0.1140068	best: 0.1139944 (196)	total: 10.4s	remaining: 15.9s
198:	learn: 0.0963098	test: 0.1139255	best: 0.1139255 (198)	total: 10.5s	remaining: 15.8s
199:	learn: 0.

284:	learn: 0.0891683	test: 0.1119662	best: 0.1119662 (284)	total: 14.3s	remaining: 10.8s
285:	learn: 0.0891440	test: 0.1119148	best: 0.1119148 (285)	total: 14.4s	remaining: 10.7s
286:	learn: 0.0890585	test: 0.1119296	best: 0.1119148 (285)	total: 14.4s	remaining: 10.7s
287:	learn: 0.0889544	test: 0.1118308	best: 0.1118308 (287)	total: 14.4s	remaining: 10.6s
288:	learn: 0.0888757	test: 0.1118151	best: 0.1118151 (288)	total: 14.5s	remaining: 10.6s
289:	learn: 0.0888564	test: 0.1118280	best: 0.1118151 (288)	total: 14.6s	remaining: 10.5s
290:	learn: 0.0887929	test: 0.1117506	best: 0.1117506 (290)	total: 14.6s	remaining: 10.5s
291:	learn: 0.0887809	test: 0.1117973	best: 0.1117506 (290)	total: 14.6s	remaining: 10.4s
292:	learn: 0.0887798	test: 0.1117944	best: 0.1117506 (290)	total: 14.7s	remaining: 10.4s
293:	learn: 0.0887610	test: 0.1117965	best: 0.1117506 (290)	total: 14.7s	remaining: 10.3s
294:	learn: 0.0887196	test: 0.1118546	best: 0.1117506 (290)	total: 14.8s	remaining: 10.3s
295:	learn

378:	learn: 0.0837309	test: 0.1108601	best: 0.1108586 (377)	total: 18.6s	remaining: 5.94s
379:	learn: 0.0837235	test: 0.1108594	best: 0.1108586 (377)	total: 18.6s	remaining: 5.89s
380:	learn: 0.0837215	test: 0.1108440	best: 0.1108440 (380)	total: 18.7s	remaining: 5.83s
381:	learn: 0.0835570	test: 0.1107472	best: 0.1107472 (381)	total: 18.7s	remaining: 5.79s
382:	learn: 0.0835045	test: 0.1107337	best: 0.1107337 (382)	total: 18.8s	remaining: 5.73s
383:	learn: 0.0833549	test: 0.1107088	best: 0.1107088 (383)	total: 18.8s	remaining: 5.68s
384:	learn: 0.0833543	test: 0.1107048	best: 0.1107048 (384)	total: 18.9s	remaining: 5.63s
385:	learn: 0.0833488	test: 0.1107034	best: 0.1107034 (385)	total: 18.9s	remaining: 5.58s
386:	learn: 0.0833114	test: 0.1107140	best: 0.1107034 (385)	total: 18.9s	remaining: 5.53s
387:	learn: 0.0833071	test: 0.1107018	best: 0.1107018 (387)	total: 19s	remaining: 5.48s
388:	learn: 0.0831880	test: 0.1106200	best: 0.1106200 (388)	total: 19s	remaining: 5.43s
389:	learn: 0.

474:	learn: 0.0800756	test: 0.1104562	best: 0.1103141 (445)	total: 22.8s	remaining: 1.2s
475:	learn: 0.0800308	test: 0.1104730	best: 0.1103141 (445)	total: 22.9s	remaining: 1.15s
476:	learn: 0.0799531	test: 0.1104329	best: 0.1103141 (445)	total: 22.9s	remaining: 1.1s
477:	learn: 0.0799333	test: 0.1104069	best: 0.1103141 (445)	total: 23s	remaining: 1.06s
478:	learn: 0.0799332	test: 0.1104067	best: 0.1103141 (445)	total: 23s	remaining: 1.01s
479:	learn: 0.0799330	test: 0.1104049	best: 0.1103141 (445)	total: 23s	remaining: 959ms
480:	learn: 0.0799137	test: 0.1104090	best: 0.1103141 (445)	total: 23.1s	remaining: 912ms
481:	learn: 0.0799110	test: 0.1104168	best: 0.1103141 (445)	total: 23.1s	remaining: 864ms
482:	learn: 0.0799050	test: 0.1104173	best: 0.1103141 (445)	total: 23.2s	remaining: 816ms
483:	learn: 0.0798808	test: 0.1103889	best: 0.1103141 (445)	total: 23.2s	remaining: 767ms
484:	learn: 0.0798806	test: 0.1103895	best: 0.1103141 (445)	total: 23.3s	remaining: 720ms
485:	learn: 0.0798

Обучим на всем лучшую модель, параметры которой мы получили на валидации:

In [64]:
cb = catboost.CatBoostClassifier(iterations=490, max_depth=7,
                                 loss_function="Logloss",
                                 custom_metric=['Logloss', 'AUC']
                                )
cb.fit(X=X.values, y=y.values, silent=True)

In [65]:
# считаем скор на test. 
# Надеюсь на вашу честность и что вы не будете валидироваться на test и честно посчитаете не нем скор
data_hour_test = pd.read_csv('./data_hw_2/data_hour_test')
data_action_test = pd.read_csv('./data_hw_2/data_action_test')

target_action = data_action_test.pivot_table(index='obj_id', values = 'target', aggfunc = ('sum', 'count')).reset_index()
target_hour = data_hour_test[['obj_id', 'target']]
target_hour.rename(columns = {'target':'hour_target'}, inplace=True)

data_hour_test = pd.merge(data_hour_test, target_action, on='obj_id')
data_action_test = pd.merge(data_action_test, target_hour, on='obj_id')

In [66]:
X_test = data_action_test.groupby('obj_id').mean()
y_test = X_test["hour_target"]
X_test = X_test.drop("hour_target", axis=1)
X_test[:4]

,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,factor_8,factor_9,...,factor_142,factor_143,factor_144,factor_145,factor_146,factor_147,factor_148,factor_149,factor_150,target
obj_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,487.0,1000.0,0.0,0.0,11.0,0.0,7.0,...,0.0,992.0,982.0,0.0,125.0,999.0,999.0,0.0,982.0,1.0
1,0.0,0.0,0.0,0.0,1000.0,0.0,NaN,2.0,0.0,0.0,...,0.0,0.0,982.0,0.0,0.0,999.0,999.0,0.0,982.0,1.0
2,136.5,51.5,0.0,0.0,1000.0,0.0,0.0,51.5,186.0,0.0,...,1.0,495.0,982.0,0.0,0.0,999.0,0.0,0.0,982.0,1.0
3,70.0,60.0,1000.0,141.0,1000.0,0.0,0.0,22.0,73.0,0.0,...,0.0,990.0,982.0,0.0,41.0,999.0,0.0,0.0,982.0,1.0


In [67]:
y_pred = cb.predict(X_test)

In [68]:
predictions = pd.DataFrame(data=y_pred, index=X_test.index, columns=['predict_target'])
predictions[:3]

,predict_target
obj_id,
0,1.0
1,1.0
2,1.0


In [69]:
data_action_test = pd.merge(data_action_test, predictions, left_on="obj_id", right_index=True)

In [70]:
# проверяем скор по action. нам нужно получить максимальный recall при precision не меньше 0.9
TP = len(data_action_test.loc[
            (data_action_test['target'] == 0) 
            & (data_action_test['hour_target'] == 1)
            & (data_action_test['predict_target'] == 1)
    ])
FP = len(data_action_test.loc[
            (data_action_test['target'] == 0) 
            & (data_action_test['hour_target'] == 0)
            & (data_action_test['predict_target'] == 1)
    ])
FN = len(data_action_test.loc[
            (data_action_test['target'] == 0) 
            & (data_action_test['hour_target'] == 1)
            & (data_action_test['predict_target'] == 0)
    ])
print('recall:{} precision:{}'.format( TP / (TP + FN), TP / (TP + FP)))

recall:0.3246645205628097 precision:0.9398952879581152


### Вторая модель

In [71]:
data_hour[:3]

,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,factor_8,factor_9,...,factor_310,factor_311,factor_312,factor_313,factor_314,factor_315,obj_id,target,count,sum
0,900.0,0.0,NaN,2.0,NaN,NaN,NaN,990.0,NaN,1000.0,...,NaN,1000.0,NaN,NaN,NaN,NaN,0,1,1,1
1,990.0,0.0,0.0,2.0,0.0,0.0,0.0,1000.0,0.0,250.0,...,NaN,1000.0,NaN,NaN,NaN,NaN,1,1,4,4
2,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,1000.0,NaN,NaN,NaN,NaN,2,1,2,0


In [73]:
for col in data_hour.columns:
    print("col = {}, {:.2f}% None values".format(col, 100 * np.sum(pd.isna(data_hour[col])) / len(data_hour[col])))

col = factor_0, 9.66% None values
col = factor_1, 0.04% None values
col = factor_2, 50.01% None values
col = factor_3, 0.04% None values
col = factor_4, 50.01% None values
col = factor_5, 50.01% None values
col = factor_6, 50.01% None values
col = factor_7, 9.66% None values
col = factor_8, 50.01% None values
col = factor_9, 0.04% None values
col = factor_10, 0.04% None values
col = factor_11, 0.04% None values
col = factor_12, 3.81% None values
col = factor_13, 11.90% None values
col = factor_14, 1.94% None values
col = factor_15, 0.00% None values
col = factor_16, 0.04% None values
col = factor_17, 0.00% None values
col = factor_18, 11.21% None values
col = factor_19, 25.08% None values
col = factor_20, 18.82% None values
col = factor_21, 3.85% None values
col = factor_22, 11.90% None values
col = factor_23, 11.90% None values
col = factor_24, 88.47% None values
col = factor_25, 9.22% None values
col = factor_26, 1.94% None values
col = factor_27, 0.02% None values
col = factor_28, 0

col = factor_259, 94.52% None values
col = factor_260, 93.59% None values
col = factor_261, 86.53% None values
col = factor_262, 93.59% None values
col = factor_263, 0.00% None values
col = factor_264, 91.11% None values
col = factor_265, 93.59% None values
col = factor_266, 88.43% None values
col = factor_267, 88.47% None values
col = factor_268, 92.38% None values
col = factor_269, 91.11% None values
col = factor_270, 92.38% None values
col = factor_271, 86.53% None values
col = factor_272, 93.59% None values
col = factor_273, 92.47% None values
col = factor_274, 99.06% None values
col = factor_275, 91.11% None values
col = factor_276, 92.38% None values
col = factor_277, 93.87% None values
col = factor_278, 93.59% None values
col = factor_279, 92.38% None values
col = factor_280, 0.00% None values
col = factor_281, 92.00% None values
col = factor_282, 92.41% None values
col = factor_283, 93.59% None values
col = factor_284, 91.11% None values
col = factor_285, 93.59% None values
col

Видим, что почти во всех колонках под названием `factor_*` бОльная часть значение -- Nan. Оставим только те, в которых пропусков очень мало

In [77]:
need_cols = []
for col in data_hour.columns:
    if np.sum(pd.isna(data_hour[col])) / len(data_hour[col]) < 0.01:
        need_cols.append(col)
print(len(need_cols))

47


In [78]:
data_hour = data_hour[need_cols]

In [204]:
data_hour[:3]

,factor_1,factor_3,factor_9,factor_10,factor_11,factor_15,factor_16,factor_17,factor_27,factor_28,...,factor_193,factor_196,factor_199,factor_263,factor_280,factor_287,obj_id,target,count,sum
0,0.0,2.0,1000.0,1000.0,0.0,0,1000.0,17,1000.0,0.0,...,0.0,0,0.0,100,0,28.0,0,1,1,1
1,0.0,2.0,250.0,750.0,0.0,0,750.0,17,1000.0,0.0,...,0.0,0,0.0,100,0,28.0,1,1,4,4
2,0.0,0.0,0.0,0.0,0.0,0,0.0,17,220.0,0.0,...,0.0,0,0.0,100,0,28.0,2,1,2,0


In [83]:
X = data_hour.drop("target", axis=1)
y = data_hour["target"]

In [84]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [190]:
cb = catboost.CatBoostClassifier(iterations=500, max_depth=6,
                                 loss_function="Logloss",
                                 custom_metric=['Logloss', 'AUC']
                                )

Валидируем и подбираем лучшие параметры. По времени этот процесс занял не очень много времени, подбирал только 2 параметра: `iterations` и `max_depth`. Этого хватило. 

In [191]:
cb.fit(X=X_train, y=y_train, eval_set=(X_val, y_val))

0:	learn: 0.4729082	test: 0.4722168	best: 0.4722168 (0)	total: 35.3ms	remaining: 17.6s
1:	learn: 0.3616811	test: 0.3605358	best: 0.3605358 (1)	total: 62.5ms	remaining: 15.6s
2:	learn: 0.2687791	test: 0.2683250	best: 0.2683250 (2)	total: 88.9ms	remaining: 14.7s
3:	learn: 0.2256286	test: 0.2248602	best: 0.2248602 (3)	total: 116ms	remaining: 14.4s
4:	learn: 0.2023615	test: 0.2017631	best: 0.2017631 (4)	total: 141ms	remaining: 14s
5:	learn: 0.1723795	test: 0.1724112	best: 0.1724112 (5)	total: 171ms	remaining: 14.1s
6:	learn: 0.1547518	test: 0.1550629	best: 0.1550629 (6)	total: 196ms	remaining: 13.8s
7:	learn: 0.1442827	test: 0.1450479	best: 0.1450479 (7)	total: 224ms	remaining: 13.8s
8:	learn: 0.1389098	test: 0.1397204	best: 0.1397204 (8)	total: 265ms	remaining: 14.5s
9:	learn: 0.1351718	test: 0.1355114	best: 0.1355114 (9)	total: 320ms	remaining: 15.7s
10:	learn: 0.1319439	test: 0.1324063	best: 0.1324063 (10)	total: 352ms	remaining: 15.6s
11:	learn: 0.1288534	test: 0.1291378	best: 0.129137

100:	learn: 0.0885241	test: 0.0969128	best: 0.0969128 (100)	total: 3.89s	remaining: 15.4s
101:	learn: 0.0882981	test: 0.0967255	best: 0.0967255 (101)	total: 3.94s	remaining: 15.4s
102:	learn: 0.0882800	test: 0.0967204	best: 0.0967204 (102)	total: 3.98s	remaining: 15.3s
103:	learn: 0.0881201	test: 0.0967800	best: 0.0967204 (102)	total: 4.01s	remaining: 15.3s
104:	learn: 0.0880810	test: 0.0967980	best: 0.0967204 (102)	total: 4.04s	remaining: 15.2s
105:	learn: 0.0879617	test: 0.0967531	best: 0.0967204 (102)	total: 4.06s	remaining: 15.1s
106:	learn: 0.0879523	test: 0.0967522	best: 0.0967204 (102)	total: 4.1s	remaining: 15.1s
107:	learn: 0.0879467	test: 0.0967482	best: 0.0967204 (102)	total: 4.15s	remaining: 15.1s
108:	learn: 0.0879413	test: 0.0967334	best: 0.0967204 (102)	total: 4.19s	remaining: 15s
109:	learn: 0.0879336	test: 0.0967231	best: 0.0967204 (102)	total: 4.23s	remaining: 15s
110:	learn: 0.0869863	test: 0.0960622	best: 0.0960622 (110)	total: 4.26s	remaining: 14.9s
111:	learn: 0.0

193:	learn: 0.0800362	test: 0.0931714	best: 0.0931714 (193)	total: 7.15s	remaining: 11.3s
194:	learn: 0.0799355	test: 0.0931810	best: 0.0931714 (193)	total: 7.2s	remaining: 11.3s
195:	learn: 0.0799127	test: 0.0931594	best: 0.0931594 (195)	total: 7.23s	remaining: 11.2s
196:	learn: 0.0799120	test: 0.0931597	best: 0.0931594 (195)	total: 7.25s	remaining: 11.2s
197:	learn: 0.0798595	test: 0.0931297	best: 0.0931297 (197)	total: 7.28s	remaining: 11.1s
198:	learn: 0.0797496	test: 0.0931200	best: 0.0931200 (198)	total: 7.32s	remaining: 11.1s
199:	learn: 0.0797493	test: 0.0931188	best: 0.0931188 (199)	total: 7.35s	remaining: 11s
200:	learn: 0.0795984	test: 0.0930883	best: 0.0930883 (200)	total: 7.41s	remaining: 11s
201:	learn: 0.0795975	test: 0.0930859	best: 0.0930859 (201)	total: 7.47s	remaining: 11s
202:	learn: 0.0795973	test: 0.0930845	best: 0.0930845 (202)	total: 7.5s	remaining: 11s
203:	learn: 0.0795963	test: 0.0930799	best: 0.0930799 (203)	total: 7.52s	remaining: 10.9s
204:	learn: 0.079562

289:	learn: 0.0764023	test: 0.0926371	best: 0.0923133 (250)	total: 11s	remaining: 7.96s
290:	learn: 0.0764010	test: 0.0926427	best: 0.0923133 (250)	total: 11s	remaining: 7.93s
291:	learn: 0.0763806	test: 0.0926647	best: 0.0923133 (250)	total: 11.1s	remaining: 7.89s
292:	learn: 0.0763801	test: 0.0926664	best: 0.0923133 (250)	total: 11.1s	remaining: 7.84s
293:	learn: 0.0763801	test: 0.0926664	best: 0.0923133 (250)	total: 11.1s	remaining: 7.79s
294:	learn: 0.0763389	test: 0.0926821	best: 0.0923133 (250)	total: 11.1s	remaining: 7.75s
295:	learn: 0.0762681	test: 0.0926975	best: 0.0923133 (250)	total: 11.2s	remaining: 7.7s
296:	learn: 0.0762678	test: 0.0926984	best: 0.0923133 (250)	total: 11.2s	remaining: 7.66s
297:	learn: 0.0762594	test: 0.0927046	best: 0.0923133 (250)	total: 11.2s	remaining: 7.62s
298:	learn: 0.0762593	test: 0.0927041	best: 0.0923133 (250)	total: 11.3s	remaining: 7.58s
299:	learn: 0.0762589	test: 0.0927031	best: 0.0923133 (250)	total: 11.3s	remaining: 7.54s
300:	learn: 0.0

383:	learn: 0.0735943	test: 0.0918770	best: 0.0917969 (371)	total: 14.3s	remaining: 4.31s
384:	learn: 0.0735942	test: 0.0918771	best: 0.0917969 (371)	total: 14.3s	remaining: 4.27s
385:	learn: 0.0735727	test: 0.0918826	best: 0.0917969 (371)	total: 14.3s	remaining: 4.23s
386:	learn: 0.0735703	test: 0.0918745	best: 0.0917969 (371)	total: 14.4s	remaining: 4.2s
387:	learn: 0.0735612	test: 0.0918693	best: 0.0917969 (371)	total: 14.4s	remaining: 4.16s
388:	learn: 0.0735611	test: 0.0918692	best: 0.0917969 (371)	total: 14.5s	remaining: 4.13s
389:	learn: 0.0734610	test: 0.0918973	best: 0.0917969 (371)	total: 14.5s	remaining: 4.09s
390:	learn: 0.0734554	test: 0.0919193	best: 0.0917969 (371)	total: 14.6s	remaining: 4.06s
391:	learn: 0.0734539	test: 0.0919236	best: 0.0917969 (371)	total: 14.6s	remaining: 4.02s
392:	learn: 0.0734420	test: 0.0919313	best: 0.0917969 (371)	total: 14.6s	remaining: 3.98s
393:	learn: 0.0734324	test: 0.0919325	best: 0.0917969 (371)	total: 14.7s	remaining: 3.95s
394:	learn:

478:	learn: 0.0712812	test: 0.0916209	best: 0.0916064 (474)	total: 17.9s	remaining: 785ms
479:	learn: 0.0712810	test: 0.0916195	best: 0.0916064 (474)	total: 18s	remaining: 748ms
480:	learn: 0.0712618	test: 0.0916114	best: 0.0916064 (474)	total: 18s	remaining: 711ms
481:	learn: 0.0712616	test: 0.0916151	best: 0.0916064 (474)	total: 18s	remaining: 673ms
482:	learn: 0.0712615	test: 0.0916140	best: 0.0916064 (474)	total: 18.1s	remaining: 636ms
483:	learn: 0.0712614	test: 0.0916132	best: 0.0916064 (474)	total: 18.1s	remaining: 598ms
484:	learn: 0.0712444	test: 0.0916008	best: 0.0916008 (484)	total: 18.1s	remaining: 560ms
485:	learn: 0.0712442	test: 0.0916007	best: 0.0916007 (485)	total: 18.2s	remaining: 523ms
486:	learn: 0.0711729	test: 0.0915975	best: 0.0915975 (486)	total: 18.2s	remaining: 486ms
487:	learn: 0.0711719	test: 0.0915996	best: 0.0915975 (486)	total: 18.2s	remaining: 448ms
488:	learn: 0.0711607	test: 0.0915916	best: 0.0915916 (488)	total: 18.3s	remaining: 411ms
489:	learn: 0.07

Этой моделью мы будем предсказывать вероятности классов. Определять какой класс соответствует какой вероятности мы будем по порогу. Иначе получить presition > 0.9 не получилось

In [242]:
cb = catboost.CatBoostClassifier(iterations=497, max_depth=10,
                                 loss_function="Logloss",
                                 custom_metric=['Logloss', 'AUC']
                                )
cb.fit(X=X_train, y=y_train, silent=True)

In [281]:
stats = []
for threshold in [0.1 * i for i in range(10)]:
    y_val_pred = (cb.predict_proba(X_val)[:, 1] > threshold).astype(int)
    
    TP = np.sum((y_val == 0) * (y_val_pred == 1))

    FP = X_val.loc[(y_val == 0) * (y_val_pred == 1), 'count'].sum() - TP

    FN = X_val.loc[(y_val == 0) * (y_val_pred == 0), 'sum'].sum()

    recall, precition = TP / (TP + FN), TP / (TP + FP)
    stats.append([threshold, recall, precition])
stats = np.array(stats)

In [285]:
def get_best_threshold(stats):
    precitions = stats[:, 2]
    good_pr = np.argwhere(precitions > 0.9)
    if len(good_pr) == 0:
        return stats[np.argmax(precitions), 0]
    good_pr = good_pr[0]
    good_stats = stats[good_pr]
    return good_stats[np.argmax(good_stats[:, 1]), 0]

In [287]:
best_threshold = get_best_threshold(stats)
best_threshold

0.8

Обучаем лучшую модель на всех данных

In [292]:
cb = catboost.CatBoostClassifier(iterations=497, max_depth=10,
                                 loss_function="Logloss",
                                 custom_metric=['Logloss', 'AUC']
                                )
cb.fit(X=X, y=y, silent=True)

In [293]:
# считаем скор на test. 
# Надеюсь на вашу честность и что вы не будете валидироваться на test и честно посчитаете не нем скор
data_hour_test = pd.read_csv('./data_hw_2/data_hour_test')
data_action_test = pd.read_csv('./data_hw_2/data_action_test')

target_action = data_action_test.pivot_table(index='obj_id', values = 'target', aggfunc = ('sum', 'count')).reset_index()
target_hour = data_hour_test[['obj_id', 'target']]
target_hour.rename(columns = {'target':'hour_target'}, inplace=True)

data_hour_test_start = pd.merge(data_hour_test, target_action, on='obj_id')
data_action_test = pd.merge(data_action_test, target_hour, on='obj_id')

In [294]:
X_test = data_hour_test_start[need_cols]
y_test = X_test["target"]
X_test = X_test.drop("target", axis=1)
X_test[:4]

,factor_1,factor_3,factor_9,factor_10,factor_11,factor_15,factor_16,factor_17,factor_27,factor_28,...,factor_173,factor_193,factor_196,factor_199,factor_263,factor_280,factor_287,obj_id,count,sum
0,0.0,0.0,0.0,0.0,0.0,0,0.0,17,330.0,0.0,...,0.0,0.0,0,0.0,100,0,28.0,0,1,1
1,0.0,0.0,0.0,0.0,0.0,0,0.0,767,230.0,0.0,...,0.0,0.0,777,0.0,800,22,28.0,1,2,2
2,0.0,2.0,500.0,1000.0,0.0,0,1000.0,17,1000.0,0.0,...,0.0,0.0,0,0.0,100,0,28.0,2,2,2
3,0.0,2.0,1000.0,1000.0,0.0,0,1000.0,17,1000.0,0.0,...,0.0,0.0,0,0.0,100,0,28.0,3,1,1


In [295]:
y_pred = (cb.predict_proba(X_test)[:, 1] > best_threshold).astype(int)

In [296]:
predictions = pd.DataFrame(data=y_pred, index=X_test.index, columns=['predict_target'])
predictions[:3]

,predict_target
0,1
1,1
2,1


In [297]:
data_hour_test = pd.merge(data_hour_test_start, predictions, left_on="obj_id", right_index=True)

In [298]:
# проверяем скор по hour. нам нужно получить максимальный recall при precision не меньше 0.9
TP = data_hour_test.loc[
            (data_hour_test['target'] == 0) 
            & (data_hour_test['predict_target'] == 1),
            'sum'].sum()

FP = data_hour_test.loc[
            (data_hour_test['target'] == 0) 
            & (data_hour_test['predict_target'] == 1),
            'count'].sum() - TP

FN = data_hour_test.loc[
            (data_hour_test['target'] == 0) 
            & (data_hour_test['predict_target'] == 0),
            'sum'].sum()

print('recall:{} precision:{}'.format( TP / (TP + FN), TP / (TP + FP)))

recall:0.02273352999016716 precision:0.9183425414364641


In [13]:
# делаем анализ факторов для модели по действиям по объектам, 
# целевую переменную которых не удалось перенести в другую систему
# т.е. мы хотим найти сильные факторы в этом факторном пространстве, которых нам не хвататает в другом факторном пространстве


In [14]:
# делаем анализ факторов для модели для часового периода по объектам, 
# целевую переменную которых не удалось перенести в другую систему
# т.е. мы хотим найти сильные факторы в этом факторном пространстве, которых нам не хвататает в другом факторном пространстве
